In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import load_model
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pandas as pd
import sklearn

In [2]:
label_encoder_gender = pickle.load(open('label_encoder_gender.pkl', 'rb'))
onehot_encoder_geography = pickle.load(open('onehot_encoder_geography.pkl', 'rb'))

In [3]:
data = pd.read_csv('Churn_Modelling.csv')

data = data.drop(['RowNumber','CustomerId','Surname'], axis=1)

data['Gender'] = label_encoder_gender.transform(data['Gender'])

geo_temp = onehot_encoder_geography.transform(data[['Geography']])
geo_df = pd.DataFrame(geo_temp.toarray(),columns=onehot_encoder_geography.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [4]:
X = data.drop(['Exited'], axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the function to create the model
def create_model(neurons=32, layers=1):
    model = Sequential()
    model.add(Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)))
    for _ in range(layers - 1):
        model.add(Dense(neurons, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Assuming a binary classification task
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Create the classifier
model = KerasClassifier(model=create_model, verbose=1)



In [13]:
from sklearn.model_selection import GridSearchCV

# Define the grid search parameters
param_grid = {
    'model__neurons': [32, 64, 128],
    'model__layers': [1, 2, 3],
    'epochs': [10, 50, 100]
}


In [14]:
# Grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=1, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

# Print best result
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

AttributeError: 'super' object has no attribute '__sklearn_tags__'